In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Your model file should be in the location: /content/drive/MyDrive/your_model.h5


Mounted at /content/drive


In [ ]:
!pip install streamlit
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 6.2 MB/s eta 0:00:00


In [ ]:
# Import libraries
import streamlit as st
import tensorflow as tf
import numpy as np
from PIL import Image
import cv2
from pyngrok import ngrok

In [ ]:
!ngrok authtoken 2nhuiMNwrunLqMynS7fZNJQyhQx_5u1XTzXwzc7BhmwKLpta9


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok

# Create an HTTP tunnel on port 8501
public_url = ngrok.connect(8501, "http")
print(f"Streamlit app will be available at: {public_url}")


Streamlit app will be available at: NgrokTunnel: "https://e5c6-34-139-162-178.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
model = tf.keras.models.load_model("/content/drive/MyDrive/best_model.keras")


In [ ]:
model.trainable = False  # Ensure model is not training


In [ ]:
# Save the Streamlit app code in a Python script
with open('streamlit_app.py', 'w') as f:
    f.write("""
import streamlit as st
import tensorflow as tf
import numpy as np
from PIL import Image
import cv2

# Load the trained model
model = tf.keras.models.load_model("/content/drive/MyDrive/best_model.keras")
model.trainable = False

# Class names for output prediction
class_names = ['AI Generated', 'Captured']
model = tf.keras.models.load_model("/content/drive/MyDrive/best_model.keras", compile=False)


# # Function to preprocess the image
# def preprocess_image(image):
#     img = np.array(image)
#     img_resized = cv2.resize(img, (180, 180))  # Resize to match training size
#     img_array = np.expand_dims(img_resized, axis=0)  # Add batch dimension
#     img_array = img_array / 255.0  # Normalize the image to [0,1]
#     return img_array

# Apply the same normalization that was used during training
def preprocess_image(image):
    image = tf.image.resize(image, (180, 180))
    image = image / 255.0  # Normalization
    return np.expand_dims(image, axis=0)  # Add batch dimension


# Streamlit UI
st.title("Image Classification: AI-Generated vs. Captured")

# Upload image
uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    # Display the uploaded image
    image = Image.open(uploaded_file)
    st.image(image, caption="Uploaded Image.", use_column_width=True)
    st.write("")

    # Preprocess the image for prediction
    img_array = preprocess_image(image)

    # Model prediction
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)[0]
    prediction_probability = predictions[0][predicted_class]

    # Display prediction
    st.write(f"**Prediction:** {class_names[predicted_class]}")
    st.write(f"**Confidence:** {prediction_probability * 100:.2f}%")

    # Show more details about prediction probabilities
    st.write("### Prediction Probabilities")
    for i, class_name in enumerate(class_names):
        st.write(f"{class_name}: {predictions[0][i] * 100:.2f}%")
""")


In [ ]:
# Run the Streamlit app in the background
!streamlit run streamlit_app.py &>/dev/null &